In [17]:
import pandas as pd

In [ ]:
player_data = pd.read_csv("data/players.csv")
player_play_data = pd.read_csv("data/player_play.csv")

print(player_play_data[pd.isna(player_play_data["pff_defensiveCoverageAssignment"])])

In [23]:
# Getting list of completions by gameId and playId
passing_plays = pd.read_csv("data/filtered_pass_plays.csv")
completions = passing_plays[passing_plays["passResult"] == "C"]
playIds = completions[["gameId", "playId"]].drop_duplicates()

# Getting tracking data at moment of pass
week_data = pd.read_csv("data/tracking_week_1.csv")
week_data = week_data[week_data["event"] == "pass_forward"]

# Filtering tracking data to only completion plays
week_data_completions = week_data.merge(playIds, on=["gameId", "playId"])

week_data_completions.to_csv("Test.csv")